In [1]:
!pip install -q ultralytics supervision tqdm onnxruntime-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.5/300.5 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.7 MB/s eta 0:00:00


In [ ]:
import cv2
from ultralytics import YOLO
import supervision as sv
from tqdm import tqdm


MODEL_PATH = "/content/drive/MyDrive/Streamlit_GUI/best.onnx"
VIDEO_PATH = "/content/drive/MyDrive/Streamlit_GUI/Inference -2.mp4"
OUTPUT_PATH = "/content/drive/MyDrive/Streamlit_GUI/output_tracked_2_720p.mp4"
TRACKER_CFG = "bytetrack.yaml"

OUT_WIDTH, OUT_HEIGHT = 1280, 720


model = YOLO(MODEL_PATH, task="detect")


cap = cv2.VideoCapture(VIDEO_PATH)
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))


fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(OUTPUT_PATH, fourcc, fps, (OUT_WIDTH, OUT_HEIGHT))


box_annotator = sv.BoxAnnotator(thickness=2)
label_annotator = sv.LabelAnnotator(text_scale=0.6, text_thickness=1)

print("🚀 Starting YOLO + ByteTrack inference at 720p...")


pbar = tqdm(total=total_frames, desc="Processing Frames", unit="frame", dynamic_ncols=True)

frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model.track(
        frame,
        persist=True,
        tracker=TRACKER_CFG,
        conf=0.1,
        iou=0.5,
        device=0
    )

    result = results[0]
    detections = sv.Detections.from_ultralytics(result)

    labels = []
    if result.boxes.id is not None:
        for i in range(len(result.boxes.cls)):
            class_id = int(result.boxes.cls[i])
            class_name = result.names[class_id]
            conf = float(result.boxes.conf[i])
            track_id = int(result.boxes.id[i])
            labels.append(f"ID {track_id} | {class_name} {conf:.2f}")
    else:
        for cls, conf in zip(result.boxes.cls, result.boxes.conf):
            labels.append(f"{result.names[int(cls)]} {conf:.2f}")

    frame = box_annotator.annotate(frame, detections=detections)
    frame = label_annotator.annotate(frame, detections=detections, labels=labels)
    frame = cv2.resize(frame, (OUT_WIDTH, OUT_HEIGHT))

    out.write(frame)

    frame_count += 1
    pbar.update(1)


pbar.close()
cap.release()
out.release()

print(f"✅ Done! Output saved to: {OUTPUT_PATH}")


🚀 Starting YOLO + ByteTrack inference at 720p...


Processing Frames:   0%|          | 0/7501 [00:00<?, ?frame/s]

Loading /content/drive/MyDrive/Streamlit_GUI/best.onnx for ONNX Runtime inference...
Using ONNX Runtime 1.23.2 with CUDAExecutionProvider

0: 512x512 1 Bus, 2 CNG_Tempos, 1 Cycle Rickshaw, 3 Jeep_SUVs, 2 Motorcycles, 10 PrivateSedan Cars, 1 Truck, 12.7ms
Speed: 2.3ms preprocess, 12.7ms inference, 2.1ms postprocess per image at shape (1, 3, 512, 512)


Processing Frames:   0%|          | 1/7501 [00:00<44:33,  2.80frame/s]


0: 512x512 1 Bus, 2 CNG_Tempos, 1 Cycle Rickshaw, 3 Jeep_SUVs, 2 Motorcycles, 10 PrivateSedan Cars, 1 Truck, 12.2ms
Speed: 3.4ms preprocess, 12.2ms inference, 1.6ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 1 Bus, 2 CNG_Tempos, 2 Jeep_SUVs, 2 Motorcycles, 10 PrivateSedan Cars, 1 Truck, 12.1ms
Speed: 2.7ms preprocess, 12.1ms inference, 1.5ms postprocess per image at shape (1, 3, 512, 512)


...


0: 512x512 3 Buss, 1 Cycle Rickshaw, 2 Jeep_SUVs, 1 Motorcycle, 7 PrivateSedan Cars, 1 Truck, 13.1ms
Speed: 3.4ms preprocess, 13.1ms inference, 4.1ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 3 Buss, 2 Jeep_SUVs, 2 Motorcycles, 6 PrivateSedan Cars, 1 Truck, 13.3ms
Speed: 3.9ms preprocess, 13.3ms inference, 1.8ms postprocess per image at shape (1, 3, 512, 512)


Processing Frames: 100%|█████████▉| 7499/7501 [08:17<00:00, 12.07frame/s]


0: 512x512 3 Buss, 2 Jeep_SUVs, 2 Motorcycles, 6 PrivateSedan Cars, 1 Truck, 13.4ms
Speed: 3.5ms preprocess, 13.4ms inference, 1.8ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 3 Buss, 2 Jeep_SUVs, 2 Motorcycles, 6 PrivateSedan Cars, 1 Truck, 12.1ms
Speed: 2.4ms preprocess, 12.1ms inference, 1.7ms postprocess per image at shape (1, 3, 512, 512)


Processing Frames: 100%|██████████| 7501/7501 [08:17<00:00, 15.07frame/s]

✅ Done! Output saved to: /content/drive/MyDrive/Streamlit_GUI/output_tracked_2_720p.mp4
